In [1]:
import torch
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F

In [2]:
class Inception(nn.Module):
    # `c1`--`c4` 是每条路径的输出通道数
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # 线路1，单1 x 1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1 x 1卷积层后接3 x 3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1 x 1卷积层后接5 x 5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3 x 3最大池化层后接1 x 1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p1, p2, p3, p4), dim=1)

In [3]:
#思考：卷积层常常只做提取特征的作用，不改变图像的高宽，但是卷积常常提取特征并做增长通道的个数，池化层一般做改变feature map的高宽的作用
B1=nn.Sequential(
    nn.Conv2d(1,64,kernel_size=7,stride=2,padding=3),nn.ReLU(),
    nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
)

In [4]:
B2=nn.Sequential(
    nn.Conv2d(64,64,kernel_size=1),nn.ReLU(),
    nn.Conv2d(64,192,kernel_size=3,padding=1),
    nn.MaxPool2d(kernel_size=3,padding=1,stride=2)
)

In [5]:
B3=nn.Sequential(
    Inception(192,64,(96,128),(16,32),32),
    Inception(256,128,(128,192),(32,96),64),
    nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
    )

In [6]:
B4=nn.Sequential(
    Inception(480,192,(96,208),(16,48),64),
    Inception(512, 160, (112, 224), (24, 64), 64),
    Inception(512, 128, (128, 256), (24, 64), 64),
    Inception(512, 112, (144, 288), (32, 64), 64),
    Inception(528, 256, (160, 320), (32, 128), 128),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)

In [7]:
B5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten())

In [8]:
net=nn.Sequential(B1,B2,B3,B4,B5,nn.Linear(1024,10))
X = torch.rand(size=(1, 1, 96, 96),dtype=torch.float32)
for layer in net:
    print(X.shape)
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

torch.Size([1, 1, 96, 96])
Sequential output shape:	 torch.Size([1, 64, 24, 24])
torch.Size([1, 64, 24, 24])
Sequential output shape:	 torch.Size([1, 192, 12, 12])
torch.Size([1, 192, 12, 12])
Sequential output shape:	 torch.Size([1, 480, 6, 6])
torch.Size([1, 480, 6, 6])
Sequential output shape:	 torch.Size([1, 832, 3, 3])
torch.Size([1, 832, 3, 3])
Sequential output shape:	 torch.Size([1, 1024])
torch.Size([1, 1024])
Linear output shape:	 torch.Size([1, 10])


In [9]:
lr, num_epochs, batch_size = 0.1, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, torch.device('cuda'))


loss 0.247, train acc 0.906, test acc 0.887
956.3 examples/sec on cuda
